In [11]:
import datetime
import json
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import time
from pathlib import Path

In [12]:
import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

In [13]:
# import sys
# sys.path.insert(1,'/data/alex/cell_seg/mae-main')

In [14]:
from tissuenetdata import SeprtSeg

AttributeError: module 'numpy.random' has no attribute 'BitGenerator'

In [ ]:
dataset_train = SeprtSeg(root_dir = '../imgs/train')
dataset_val = SeprtSeg(root_dir = '../imgs/val')

In [ ]:
dataset_train.__getitem__(0)['tissuelabel'] > 0

In [ ]:
batch_size = 8
num_workers = 10
pin_mem = True

In [ ]:
# dataset_train.__getitem__(0)
data_loader_train = torch.utils.data.DataLoader(
    dataset_train, shuffle=True,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_mem,
    drop_last=True,
)

data_loader_val = torch.utils.data.DataLoader(
    dataset_val, shuffle=True,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_mem,
    drop_last=False
)

In [ ]:
from swinunet import SwinUnet
# from models_vit_seg_2 import vit_seg_large
# import segmentation_models_pytorch as smp

In [ ]:
# net = smp.UnetPlusPlus(
#     encoder_name = 'resnext50_32x4d',
#     in_channels = 2,
#     # decoder_attention_type = 'scse',
#     decoder_use_batchnorm = True,
#     activation = 'sigmoid',
#     classes=1
# )

In [ ]:
net = SwinUnet(
    num_classes=1
)

In [ ]:
# net = vit_seg_large()

In [ ]:
out = net(
    torch.randn(10,2,256,256)
)

In [ ]:
out.size()

In [ ]:
out.min()

In [ ]:
device = 'cuda'
device = torch.device(device)

In [ ]:
net.to(device)
###
print("to {}".format(device))

In [ ]:
lr = 1e-3* 32/256
min_lr = 1e-6
weight_decay = 0.05
layer_decay = 0.75

In [ ]:
# import util.lr_decay as lrd
from torchinfo import summary

In [ ]:
summary(net, (10,2,256,256))

In [ ]:
# len(net.blocks) + len(net.seg_blocks)

In [ ]:
def param_groups_lrd(model, weight_decay=0.05, no_weight_decay_list=[], layer_decay=.75):
    """
    Parameter groups for layer-wise lr decay
    Following BEiT: https://github.com/microsoft/unilm/blob/master/beit/optim_factory.py#L58
    """
    param_group_names = {}
    param_groups = {}

    num_layers = 32 + 1#len(model.swin_unet.layers) + 1

    layer_scales = list(layer_decay ** (num_layers - i) for i in range(num_layers + 1))

    for n, p in model.named_parameters():
        if not p.requires_grad:
            continue

        # no decay: all 1D parameters and model specific ones
        if p.ndim == 1 or n in no_weight_decay_list:
            g_decay = "no_decay"
            this_decay = 0.
        else:
            g_decay = "decay"
            this_decay = weight_decay
            
        layer_id = get_layer_id_for_vit(n, num_layers)
        group_name = "layer_%d_%s" % (layer_id, g_decay)

        if group_name not in param_group_names:
            this_scale = layer_scales[layer_id]

            param_group_names[group_name] = {
                "lr_scale": this_scale,
                "weight_decay": this_decay,
                "params": [],
            }
            param_groups[group_name] = {
                "lr_scale": this_scale,
                "weight_decay": this_decay,
                "params": [],
            }

        param_group_names[group_name]["params"].append(n)
        param_groups[group_name]["params"].append(p)

    # print("parameter groups: \n%s" % json.dumps(param_group_names, indent=2))

    return list(param_groups.values())

def get_layer_id_for_resnext(name, num_layers):
    """
    Assign a parameter with its layer id
    Following BEiT: https://github.com/microsoft/unilm/blob/master/beit/optim_factory.py#L33
    """
    # print(name)
    if name in ['cls_token', 'pos_embed']:
        return 0
    elif 'patch_embed' in name:
        return 0
    elif 'layer' in name:
        nmlst = name.split('.')
        return int(nmlst[1][-1])*5 + int(nmlst[2])
    else:
        return num_layers


def get_layer_id_for_vit(name, num_layers):
    """
    Assign a parameter with its layer id
    Following BEiT: https://github.com/microsoft/unilm/blob/master/beit/optim_factory.py#L33
    """
    print(name)
    if name in ['cls_token', 'pos_embed']:
        return 0
    elif 'patch_embed' in name:
        return 0
    elif 'blocks' in name:
        # print(name.split("."))
        return int(name.split('.')[2]) + 1
    else:
        return num_layers

In [ ]:
param_groups = param_groups_lrd(
    net, 
    weight_decay,
        # no_weight_decay_list=model_without_ddp.no_weight_decay(),
    layer_decay=layer_decay
)
optimizer = torch.optim.AdamW(param_groups, lr=lr)

In [ ]:
## Only for MAE vit:
# from util.pos_embed import interpolate_pos_embed

# finetune = '../p65maskr/checkpoint-249.pth'

# checkpoint = torch.load(finetune, map_location='cpu')

# print("Load pre-trained checkpoint from: %s" % finetune)
# checkpoint_model = checkpoint['model']
# state_dict = net.state_dict()
# for k in ['head.weight', 'head.bias']:
#     if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
#         print(f"Removing key {k} from pretrained checkpoint")
#         del checkpoint_model[k]

#         # interpolate position embedding
# interpolate_pos_embed(net, checkpoint_model)

#         # load pre-trained model
# msg = net.load_state_dict(checkpoint_model, strict=False)
# print(msg)

In [ ]:
from util.misc import NativeScalerWithGradNormCount as NativeScaler
import util.misc as misc
from timm.data import Mixup
import math
import sys
from typing import Iterable, Optional
# import util.lr_sched as lr_sched

In [ ]:
loss_scaler = NativeScaler()

In [ ]:
# del topo_loss

In [ ]:
from tp_loss import topo_loss

In [ ]:
criterion = topo_loss()

In [ ]:
import math
from pathlib import Path
from util.misc import is_main_process

def adjust_learning_rate(optimizer, epoch, num_epochs=250, warmup_epochs=10, lr=lr, min_lr=min_lr):
    """Decay the learning rate with half-cycle cosine after warmup"""
    # print(num_epochs)
    if epoch < warmup_epochs:
        lr = lr * epoch / warmup_epochs 
    else:
        lr = min_lr + (lr - min_lr) * 0.5 * \
            (1. + math.cos(math.pi * (epoch - warmup_epochs) / (num_epochs - warmup_epochs)))
    for param_group in optimizer.param_groups:
        if "lr_scale" in param_group:
            param_group["lr"] = lr * param_group["lr_scale"]
        else:
            param_group["lr"] = lr
    return lr

def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)

def save_model(output_dirnm, epoch, model, model_without_ddp, optimizer, loss_scaler):
    output_dir = Path(output_dirnm)
    epoch_name = str(epoch)
    if loss_scaler is not None:
        checkpoint_paths = [output_dir / ('checkpoint-%s.pth' % epoch_name)]
        for checkpoint_path in checkpoint_paths:
            to_save = {
                'model': model_without_ddp.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch,
                'scaler': loss_scaler.state_dict()
            }

            save_on_master(to_save, checkpoint_path)
    else:
        client_state = {'epoch': epoch}
        model.save_checkpoint(save_dir=output_dinmr, tag="checkpoint-%s" % epoch_name, client_state=client_state)

In [ ]:
def train_one_epoch(model: torch.nn.Module, criterion: torch.nn.Module,
                    data_loader: Iterable, optimizer: torch.optim.Optimizer,
                    device: torch.device, epoch: int, loss_scaler, max_norm: float = 0,
                    mixup_fn: Optional[Mixup] = None, log_writer=None,
                    args=None):
    model.train(True)
    metric_logger = misc.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('mse', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    metric_logger.add_meter('mmt', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 20

    accum_iter = 1

    optimizer.zero_grad()

    if log_writer is not None:
        print('log_dir: {}'.format(log_writer.log_dir))
        
    for item in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        print(item[1].keys())
        break
    
    
    for data_iter_step, datum in enumerate(metric_logger.log_every(data_loader, print_freq, header)):

        # we use a per iteration (instead of per epoch) lr scheduler
        if data_iter_step % accum_iter == 0:
            adjust_learning_rate(optimizer, data_iter_step / len(data_loader) + epoch)
        
        samples, targets = datum['image'], datum['ind']
        samples = samples.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        if mixup_fn is not None:
            samples, targets = mixup_fn(samples, targets)

        with torch.cuda.amp.autocast():
            outputs = model(samples)
            loss, mse_v, mmt_v = criterion(outputs, targets)

        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            sys.exit(1)

        loss /= accum_iter
        loss_scaler(loss, optimizer, clip_grad=max_norm,
                    parameters=model.parameters(), create_graph=False,
                    update_grad=(data_iter_step + 1) % accum_iter == 0)
        if (data_iter_step + 1) % accum_iter == 0:
            optimizer.zero_grad()

        torch.cuda.synchronize()

        metric_logger.update(loss=loss_value)
        metric_logger.update(mse=mse_v)
        metric_logger.update(mmt=mmt_v)
        
        min_lr = 10.
        max_lr = 0.
        for group in optimizer.param_groups:
            min_lr = min(min_lr, group["lr"])
            max_lr = max(max_lr, group["lr"])

        metric_logger.update(lr=max_lr)

        loss_value_reduce = misc.all_reduce_mean(loss_value)
        mse_reduce = misc.all_reduce_mean(mse_v)
        mmt_reduce = misc.all_reduce_mean(mmt_v)
        if log_writer is not None and (data_iter_step + 1) % accum_iter == 0:
            """ We use epoch_1000x as the x-axis in tensorboard.
            This calibrates different curves when batch size changes.
            """
            epoch_1000x = int((data_iter_step / len(data_loader) + epoch) * 1000)
            log_writer.add_scalar('loss', loss_value_reduce, epoch_1000x)
            log_writer.add_scalar('lr', max_lr, epoch_1000x)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

@torch.no_grad()
def evaluate(data_loader,criterion, model, device):

    metric_logger = misc.MetricLogger(delimiter="  ")
    header = 'Validation:'

    # switch to evaluation mode
    model.eval()

    for batch in metric_logger.log_every(data_loader, 10, header):
        images = batch['image']
        targets = batch['ind']
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        # compute output
        with torch.cuda.amp.autocast():
            output = model(images)
            loss, mse_v, mmt_v = criterion(output, targets)
            
        batch_size = images.shape[0]
        metric_logger.update(loss=loss.item())
        metric_logger.update(mse=mse_v)
        metric_logger.update(mmt=mmt_v)
        

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print('loss {losses.global_avg:.3f}'
          .format(losses=metric_logger.loss))

    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

In [ ]:
out_dir = '../experiments/nuc_test_4_17_2023'

In [ ]:
os.makedirs(out_dir, exist_ok=True)
log_writer = SummaryWriter(log_dir=out_dir)

In [ ]:
n_parameters = sum(p.numel() for p in net.parameters() if p.requires_grad)

In [ ]:
config_dic = {
    'lr' : 1e-3* 32/256,
    'min_lr' : 1e-6,
    'weight_decay' : 0.05,
    'layer_decay' : 0.75
}   
with open(os.path.join(out_dir, "config_model.txt"),mode="a", encoding="utf-8") as f:
    f.write(json.dumps(config_dic) + "\n")
          
for epoch in range(0, 250):
    train_stats = train_one_epoch(
        net, criterion, data_loader_train,
        optimizer, device, epoch, loss_scaler,
        None, None,
        log_writer=log_writer,
    )

    save_model(
        output_dirnm= out_dir, model=net, model_without_ddp=net, optimizer=optimizer,
        loss_scaler=loss_scaler, epoch=epoch)
    test_stats = evaluate(data_loader_val, criterion, net, device)
    log_writer.add_scalar('val/val_loss', test_stats['loss'], epoch)
    log_writer.add_scalar('val/mse_loss', test_stats['mse'], epoch)
    log_writer.add_scalar('val/mmt_loss', test_stats['mmt'], epoch)
    log_writer.add_scalar('train/train_loss', train_stats['loss'], epoch)
    log_writer.add_scalar('train/mse_loss', train_stats['mse'], epoch)
    log_writer.add_scalar('train/mmt_loss', train_stats['mmt'], epoch)
    
    
    log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                        **{f'test_{k}': v for k, v in test_stats.items()},
                        'epoch': epoch,
                        'n_parameters': n_parameters}
    
    if log_writer is not None:
        log_writer.flush()
    with open(os.path.join( out_dir, "log.txt"), mode="a", encoding="utf-8") as f:
        f.write(json.dumps(log_stats) + "\n")
